In [ ]:
import numpy as np
import random
import html
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import rapidfuzz
import tqdm
import matplotlib as plt
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
df_employees = pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Employees\Apollo\dataset_leads-finder_2025-12-10_16-53-58-541.csv")
df_employees.head()

,city,company_annual_revenue,company_annual_revenue_clean,company_city,company_country,company_description,company_domain,company_founded_year,company_full_address,company_linkedin,company_linkedin_uid,company_name,company_phone,company_postal_code,company_size,company_state,company_street_address,company_technologies,company_total_funding,company_total_funding_clean,company_website,country,email,first_name,full_name,functional_level,headline,industry,job_title,keywords,last_name,linkedin,mobile_number,personal_email,seniority_level,state
0,NaN,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Croatia,damir.klarin@dahuatech.com,Damir,Damir Klarin,sales,NaN,Information Technology & Services,Business Development Manager,"dvrs, nvr, ivs, ip cameras, security solution ...",Klarin,http://www.linkedin.com/in/damir-klarin-8496283,NaN,NaN,manager,NaN
1,Dubai,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,United Arab Emirates,NaN,Ali,Ali Kloub,sales,Regional Director (mena)\r\nbusiness Growth | ...,Information Technology & Services,Regional Sales Director (mena),"dvrs, nvr, ivs, ip cameras, security solution ...",Kloub,http://www.linkedin.com/in/ali-kloub-b15b2657,NaN,alikloub@hotmail.com,director,Dubai
2,Jeddah,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Saudi Arabia,yahya.asfari@dahuatech.com,Yahya,Yahya Asfari,engineering,Director Of Business Development | Ai Surveill...,Information Technology & Services,Director Of Business Development | Ai Surveill...,"dvrs, nvr, ivs, ip cameras, security solution ...",Asfari,http://www.linkedin.com/in/yahya1asfari,NaN,NaN,director,Makkah Province
3,Bogota,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Colombia,ruben.trujillo@dahuatech.com,Jose,Jose Ruben Trujillo,sales,Business Development Director,Information Technology & Services,Business Development Director,"dvrs, nvr, ivs, ip cameras, security solution ...",Ruben Trujillo,http://www.linkedin.com/in/jruben-trujillo,NaN,NaN,director,Bogota
4,Beirut,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Lebanon,NaN,Rodolphe,Rodolphe Kassouf,sales,Director Of Business Development,Information Technology & Services,Director Of Business Development,"dvrs, nvr, ivs, ip cameras, security solution ...",Kassouf,http://www.linkedin.com/in/rodolphe-kassouf,NaN,

In [9]:
df_employees_c = pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\employees_cleaned\df_employees_cleaned.csv")
df_employees_c.head()

,city,company_founded_year,company_linkedin_uid,company_name,company_size,country,email,full_name,functional_level,headline,industry,job_title,linkedin,seniority_level,state
0,NaN,2001,2845844,Dahua Technology Co. Ltd,23000,Croatia,damir.klarin@dahuatech.com,Damir Klarin,sales,NaN,Information Technology & Services,Business Development Manager,http://www.linkedin.com/in/damir-klarin-8496283,manager,NaN
1,Dubai,2001,2845844,Dahua Technology Co. Ltd,23000,United Arab Emirates,NaN,Ali Kloub,sales,Regional Director (mena)\r\nbusiness Growth | ...,Information Technology & Services,Regional Sales Director (mena),http://www.linkedin.com/in/ali-kloub-b15b2657,director,Dubai
2,Jeddah,2001,2845844,Dahua Technology Co. Ltd,23000,Saudi Arabia,yahya.asfari@dahuatech.com,Yahya Asfari,engineering,Director Of Business Development | Ai Surveill...,Information Technology & Services,Director Of Business Development | Ai Surveill...,http://www.linkedin.com/in/yahya1asfari,director,Makkah Province
3,Bogota,2001,2845844,Dahua Technology Co. Ltd,23000,Colombia,ruben.trujillo@dahuatech.com,Jose Ruben Trujillo,sales,Business Development Director,Information Technology & Services,Business Development Director,http://www.linkedin.com/in/jruben-trujillo,director,Bogota
4,Beirut,2001,2845844,Dahua Technology Co. Ltd,23000,Lebanon,NaN,Rodolphe Kassouf,sales,Director Of Business Development,Information Technology & Services,Director Of Business Development,http://www.linkedin.com/in/rodolphe-kassouf,director,Beirut Governorate
